In [7]:
import numpy as np
import cv2 as cv
import glob


################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (7,10)
frameSize = (672,376)
# chessboardSize = (10,7)
# frameSize = (640,480)


# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 20
objp = objp * size_of_chessboard_squares_mm

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane. 
imgpointsR = [] # 2d points in image plane.


imagesLeft = sorted(glob.glob('./left_calibration_images/*.jpg'))
imagesRight = sorted(glob.glob('./right_calibration_images/*.jpg'))

for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        cv.waitKey(1000)


cv.destroyAllWindows()




############## CALIBRATION #######################################################

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))



########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)




########## Stereo Rectification #################################################

rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv.CV_16SC2)
stereoMapR = cv.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv.CV_16SC2)

print("Saving parameters!")
cv_file = cv.FileStorage('stereoMap.xml', cv.FILE_STORAGE_WRITE)

print("Left Camera Intrinsic Matrix:\n", newCameraMatrixL)
print("Left Camera Distortion Coefficients:\n", distL)
print("Right Camera Intrinsic Matrix:\n", newCameraMatrixR)
print("Right Camera Distortion Coefficients:\n", distR)
print("Rotation Matrix (R):\n", rot)
print("Translation Vector (T):\n", trans)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()


Saving parameters!
Left Camera Intrinsic Matrix:
 [[488.83389282   0.         324.91179122]
 [  0.         408.83706665 182.07088095]
 [  0.           0.           1.        ]]
Left Camera Distortion Coefficients:
 [[-8.58310125e-01  5.89170830e+00  4.13658181e-03  7.00863211e-02
  -1.31346787e+01]]
Right Camera Intrinsic Matrix:
 [[ 58.80132675   0.         629.44845918]
 [  0.          87.18722534  57.0522326 ]
 [  0.           0.           1.        ]]
Right Camera Distortion Coefficients:
 [[-1.65371028  3.32709555 -0.06483818 -0.01288325 -2.79309744]]
Rotation Matrix (R):
 [[ 0.05527149  0.66300806 -0.74656907]
 [-0.34446445  0.71446852  0.60899834]
 [ 0.9371709   0.22350626  0.26787245]]
Translation Vector (T):
 [[17077.63165815]
 [-2108.0907344 ]
 [12865.87112664]]


In [8]:
baseline = np.linalg.norm(trans)
baseline

21485.3482504501

In [3]:
desired_focal_length=560
reference_focal_length=519
scaling_factor=desired_focal_length/reference_focal_length
new_intrinsic_matrix_reference_camera = scaling_factor * newCameraMatrixR

print(new_intrinsic_matrix_reference_camera)

[[184.33849651   0.         369.45916623]
 [  0.         120.08859506 369.96986664]
 [  0.           0.           1.07899807]]
